<a href="https://colab.research.google.com/github/Tatsuro0726/chemoinfomatics/blob/main/deepchem/Tutorial7_Deeper_on_Molecule_Featurizations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 分子の特徴化について深く考える
分子データに対しての機械学習で重要なステップの１つは、データを学習アルゴリズムに適した形に変換することである。  
分子をベクトルやテンソルに変換するなど、様々な方法があり、どの変換方法を選択するかは、対象の問題に依存する部分がある。
例として既に、FingerprintとConvMolオブジェクトの２つを見た。

#### Setup

In [6]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3490  100  3490    0     0  18967      0 --:--:-- --:--:-- --:--:-- 19071


all packages are already installed


# conda environments:
#
base                  *  /root/miniconda



In [7]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

'2.5.0.dev'

#### Featurizer
Deepchemでは、分子を特徴ベクトルに変換する方法がFeaturizerオブジェクトで定義されている。

1. MoleculeNetデータを使用する場合
    - 使用するメソッドの文字列(ECFP,GraphConv)を渡すだけ。
2. 分子に直接適用することもできる。(以下に例を示す)


In [8]:
import deepchem as dc

featurizer = dc.feat.CircularFingerprint()
print(featurizer(['CC', 'CCC','CCO']))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


#### RDkitDescriptors
RDKitDescriptorsは、RDKitを使用して記述子のリストの値を計算することで、分子を機能させます。これらは基本的な物理的・化学的性質であり、分子量、極性表面積、水素結合ドナーとアクセプターの数などです。これは、詳細な分子構造よりも、これらの高レベルの特性に依存するものを予測するのに最も有用です。

In [9]:
# RDkitでの分子記述子
rdkit_featurizer = dc.feat.RDKitDescriptors()
features = rdkit_featurizer(['CCC'])[0]
for feature, descriptor in zip(features[:10], rdkit_featurizer.descriptors):
    print(descriptor, feature)

MaxEStateIndex 2.125
MinEStateIndex 1.25
MaxAbsEStateIndex 2.125
MinAbsEStateIndex 1.25
qed 0.3854706587740357
MolWt 44.097
HeavyAtomMolWt 36.033
ExactMolWt 44.062600255999996
NumValenceElectrons 20.0
NumRadicalElectrons 0.0


In [10]:
print('The number of descriptors present is:', len(features))

The number of descriptors present is: 208


#### Weave Featurizer および MolGraphConvFeaturizer

Deepchemは、多くの異なるグラフベースのモデルをサポートしている。
- WeaveFeaturizer
- MolGraphConvFeaturizer  

上記の方法は分子を特定のモデルで使用される異なるタイプのPythonオブジェクトに変換。グラフベースのモデルを使用するときはreferenceをチェックすること。

#### クーロン行列
分子の立体配座のための１つの特徴化方法。
原子数がNの場合、N×Nの行列となり、各要素は２つの
原子間の静電相互作用の強さを表す。これは、原子上の電荷とそれらの間の距離の情報を含んでいる。

このFeaturizerを適用するには、分子の立体配座が必要になる。ConformerGeneratorクラスを使用する。RDkit moleculeを受け取り、エネルギーを最小化したコンフォーマーのセットを生成し、お互いに有意に異なるものだけを含むようにセットを刈り込みます。

In [12]:
from rdkit import Chem

generator = dc.utils.ConformerGenerator(max_conformers=5)
propane_mol = generator.generate_conformers(Chem.MolFromSmiles('CCC'))
print('Number of available conformers for propane: {}'.format(len(propane_mol.GetConformers())))

Number of available conformers for propane: 1


In [13]:
butane_mol = generator.generate_conformers(Chem.MolFromSmiles('CCCC'))
print("Number of available conformers for butane: ", len(butane_mol.GetConformers()))

Number of available conformers for butane:  3


In [14]:
coulomb_mat = dc.feat.CoulombMatrix(max_atoms=20)
features = coulomb_mat(propane_mol)
print(features)

[[[36.8581052  12.48684429  7.5619687   2.85945193  2.85804514
    2.85804556  1.4674015   1.46740144  0.91279491  1.14239698
    1.14239675  0.          0.          0.          0.
    0.          0.          0.          0.          0.        ]
  [12.48684429 36.8581052  12.48684388  1.46551218  1.45850736
    1.45850732  2.85689525  2.85689538  1.4655122   1.4585072
    1.4585072   0.          0.          0.          0.
    0.          0.          0.          0.          0.        ]
  [ 7.5619687  12.48684388 36.8581052   0.9127949   1.14239695
    1.14239692  1.46740146  1.46740145  2.85945178  2.85804504
    2.85804493  0.          0.          0.          0.
    0.          0.          0.          0.          0.        ]
  [ 2.85945193  1.46551218  0.9127949   0.5         0.29325367
    0.29325369  0.21256978  0.21256978  0.12268391  0.13960187
    0.13960185  0.          0.          0.          0.
    0.          0.          0.          0.          0.        ]
  [ 2.85804514  1.458

/usr/local/lib/python3.6/dist-packages/deepchem/feat/molecule_featurizers/coulomb_matrices.py:138: RuntimeWarning: divide by zero encountered in true_divide
  m = np.outer(z, z) / d


上記のクーロン行列について、多くの要素が"0"であることに注意してください。複数の分子をまとめて変換するには、分子の原子数が異なっていても、クーロン行列はすべて同じ大きさにする必要がある。

ここでは、max_atoms=20と指定しているので、返される行列のサイズは(20,20)となる。変換した分子は11個の原子しかないので、11×11の部分行列だけが非ゼロ。

#### クーロン行列Eig
クーロン行列の重要な特徴は、原子間距離や原子番号が変化しないため、分子の回転や並進に対して不変であること。  
このような対称性を考慮することで、学習が容易になる。

分子を回転させても物性は変化しない。Featurizationを変えた場合、モデルは回転が重要でないことを強制的に学習するべきだが、Featurizationが不変であれば、モデルは自動的にこの性質を獲得する。

クーロン行列は、原子のインデックスの順序については変化してします。

分子の物理的性質は原子の順序性には依存しないが、クーロン行列は依存する。この問題に対処するため、クーロン行列の固有スペクトルを利用して、原子のインデックスのランダムな並びに不変である、CoulumbMatrixEig featurizerを実装された。

このFeaturizationの欠点は、含まれる情報量が少ないこと(N×Nの行列の代わりに、N個の固有値をしようするため)。そのため、モデルの学習が制限されてしまう。

CoulombMatrixEigは，CoulombMatrixを継承し，まず分子の異なるコンフォーマルに対するクーロン行列を計算し，次に各クーロン行列の固有値を計算することで分子を高精度化します．これらの固有値は，分子間の原子数のばらつきを考慮してパディングされます．




In [15]:
# クーロン行列の固有値
coulomb_mat_eig = dc.feat.CoulombMatrixEig(max_atoms=20)
features = coulomb_mat_eig(propane_mol)
print(features)

[[60.07620303 29.62963149 22.75497781  0.5713786   0.28781332  0.28548338
   0.27558187  0.18163794  0.17460999  0.17059719  0.16640098  0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]]


/usr/local/lib/python3.6/dist-packages/deepchem/feat/molecule_featurizers/coulomb_matrices.py:138: RuntimeWarning: divide by zero encountered in true_divide
  m = np.outer(z, z) / d
